# Lectura de la Información
En este notebook nos enfocaremos en extraer la data de los archivos comprimidos y unificar las fuentes de datos. Muchas de los métodos/funciones utilizadas, serán cargadas de un archivo utils.py previamente generado para evitar sobrecarga de código en este notebook.

### Cargar librerías necesarias

In [1]:
from utils import *
import shutil
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Listar archivos comprimidos y extraer archivos

In [3]:
files=[i for i in os.listdir("../") if i.endswith(".zip")] ## ../ hace referencia a un nivel atrás al actual, en donde se encuentran los archivos .zip
_=[shutil.unpack_archive(f'../{i}', f'../data/') for i in files]

In [4]:
##extraer paths de cada archivo csv
filepaths=get_csv_filepaths([i.replace(".zip","") for i in files])

### Para leer las fuentes de datos se definirán grupos de archivos denotados por la variable filegroups

In [5]:
filegroups=['personajuridica','personanatural','produccion','recaudo','siniestro']     
data={}
for filegp in filegroups:
    data[filegp]=read_by_filegroup(filegp,filepaths,save_output=True)

In [7]:
##leer archivos pickle

data={}
for filegp in filegroups:
    data[filegp]=pd.read_pickle(f"{filegp}.pickle",compression="gzip")

## Esquema de generación de la variable objetivo: churn

- Se deben explorar los campos que permiten unir las diversas fuentes de información, para ello deberá explorarse el código de asegurado o el número de la póliza
- Debido a que solo se tiene un año de historia, debemos tomar un periodo base y observar 12 meses después el comportamiento de los asegurados, para encontrar si renovaron o no (churn) su poliza. A través de este análisis generaremos la variable objetivo. Esto puede observarse en el siguiente gráfico. 

![img](images/caso_uso_previsora.jpg)




In [79]:
##detectar columnas tipo fecha

date_cols=[i for i in data['produccion'].columns.values if "fecha" in i.lower()]

## crear una copia del dataframe para generar reproducibilidad
data_prod=data['produccion'].drop_duplicates().copy()

def col_to_dateutc(column):
    return pd.to_datetime(column,errors="coerce").dt.strftime('%m-%Y')

for col in date_cols:
    data_prod.loc[:,col]=col_to_dateutc(data_exp['FechaEmision__c'])

print("Total filas antes de remover duplicados: ",data['produccion'].shape[0])
print("Total filas después de remover duplicados: ",data_prod.shape[0])

Total filas antes de remover duplicados:  1054290
Total filas después de remover duplicados:  1003437


In [117]:
fecha_inicio="02-2021" ##fecha para iniciar el monitoreo de polizas un año después
fecha_fin="02-2022"  ##fecha para finalizar el monitoreo de las pólizas
emisiones_inicio=data_prod.query(f'FechaEmision__c=="{fecha_inicio}"')
emisiones_final=data_prod.query(f'FechaEmision__c=="{fecha_fin}"')
print(f"Total emisiones en {fecha_inicio} : {len(set(emisiones_inicio.NumeroPoliza__c.values))} polizas")
print(f"Total emisiones al finalizar el monitoreo en {fecha_fin} : {len(set(emisiones_final.NumeroPoliza__c.values))} polizas")

Total emisiones en 02-2021 : 72753 polizas
Total emisiones al finalizar el monitoreo en 02-2022 : 52936 polizas
